In [1]:
# Obtiene la curva de distribución acumulada del modelo.

import os

import numpy as np
import pandas as pd

import xarray as xr

In [14]:
vars = [ "Pressure", "Relative_Humidity", "Temperature",
    "Wind_Direction", "Wind_Speed", "DNI", "GHI", "UVHI",
    #"Surface_Albedo", "Precipitable_Water"
    ]
dims = [ "XTIME", "south_north", "west_east" ]

model = False
if model:
    path_d = "../temp/TMY/quantile_vars/"
    path_r = "../temp/TMY/CDF_vars_model/"
else:
    path_d = "../temp/NSRDB/quantile_vars/"
    path_r = "../temp/NSRDB/CDF_vars_model/"

    #vars += [ "UV_A", "UV_B" ]

# Iteramos para todas las variables a mapear.
for v in vars:
    print(f"\n{v}")

    # Iteramos para todos los archivos.
    files = os.listdir( path_d + v + "/" )
    files.sort()
    if ".DS_Store" in files: files.remove(".DS_Store")
    for f in files:
        print( f"{f[-5:-3]}", end = "," )

        with xr.open_dataset(path_d + v + "/" + f) as ds:
            ds_2 = ds.drop_vars( list(ds.keys()) ).copy() 
            df = ds.to_dataframe().drop( ["XLAT", "XLONG"], axis = 1 )

            # A las variables de radiación les quitamos los ceros.
            if v in vars[-3:]: df = df[ df>0 ].dropna()

            latitude = df.index.get_level_values(dims[1]).unique()
            longitude = df.index.get_level_values(dims[2]).unique()

            # Obtenemos la curva de distribución acumulada para cada celda.
            df["q_model"] = 0.0
            for lat in latitude:
                for lon in longitude:
                    df_c = df.xs( (slice(None), lat, lon) ).sort_values(v)
                    df_c["q_model"] = np.linspace(
                        1/df_c.shape[0], 1, df_c.shape[0] )
                    df.loc[ (slice(None), lat, lon), "q_model"
                        ] = df_c["q_model"].values

            ds_2["q_model"] = df["q_model"].to_xarray()
            ds["q_model"] = ( dims, ds_2["q_model"].values )
            # Guardamos el archivo.
            if not os.path.exists( path_r + v + "/"):
                os.mkdir( path_r + v + "/" )
            ds.to_netcdf( path_r + v + "/" + f, mode = "w" )


Pressure
54,
Relative_Humidity
54,
Temperature
54,
Wind_Direction
54,
Wind_Speed
54,
DNI
54,
GHI
54,
UVHI
54,